# 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline


import time
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from sklearn.preprocessing import Normalizer
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
import random
#import shap
#import optuna

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor

import random
import os

import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/sample_submission.csv')

In [ ]:
train.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)
test.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)

### train 전처리

In [ ]:
# 스크류 삽입깊이 단차
train['max-min_X'] = train[['X_19','X_20','X_21','X_22']].max(axis=1) - train[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
train['slope_X'] = train[['X_38','X_39','X_40']].max(axis = 1) - train[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
train['SMT_X'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
# 비슷한 종류들끼리 평균
train['X1~X6'] =  train[['X_01','X_06']].mean(axis=1)
train['X14~X18'] = train[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
train['X19~X22'] = train[['X_19','X_20','X_21','X_22']].mean(axis=1)
train['X24~X29'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
train['X30~X32'] = train[['X_30','X_32']].mean(axis=1)
train['X31~X33'] = train[['X_31','X_33']].mean(axis=1)
train['X38~X40'] = train[['X_38','X_39','X_40']].mean(axis=1)
train['X41~X45'] = train[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
train['X50~X56'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
# 단차
train['con1_X'] = train['X_12'] - train['X_24']
train['con2_X'] = train['X_12'] - train['X_25']
train['con3_X'] = train['X_12'] - train['X_26']
train['con4_X'] = train['X_12'] - train['X_27']
train['con5_X'] = train['X_12'] - train['X_28']
train['con6_X'] = train['X_12'] - train['X_29']

In [ ]:
# 중복으로 넣기
train['nd_area_X1'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X1'] = train['X_09']*train['X_07']/train['X_03']

train['nd_area_X2'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X2'] = train['X_09']*train['X_07']/train['X_03']

In [ ]:
train = train.replace(train['X_24'].min(), train['X_24'].mean())

In [ ]:
# 무게 / 면적
train['X3/X7'] = train['X_03']/ train['X_07']

# 스크류 깊이 평균
train['X19X30'] = train[['X_19','X_30']].mean(axis=1)
train['X20X31'] = train[['X_20','X_31']].mean(axis=1)
train['X21X32'] = train[['X_21','X_32']].mean(axis=1)
train['X22X33'] = train[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
train['X14~X18_std'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
train['X19~X22_std'] = train[['X_19','X_20','X_21','X_22']].std(axis=1)
train['X24~X29_std'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

train['X38~X40_std'] = train[['X_38','X_39','X_40']].std(axis=1)

train['X41~X45_std'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
train['X50~X56_std'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
# 중앙값
train['X14~X18_mid'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
train['X19~X22_mid'] = train[['X_19','X_20','X_21','X_22']].median(axis=1)
train['X24~X29_mid'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

train['X38~X40_mid'] = train[['X_38','X_39','X_40']].median(axis=1)

train['X41~X45_mid'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
train['X50~X56_mid'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
# 튀는값 평균으로 대치
mean_X_30 = train[(train['X_30']<1.7) & (train['X_30']>1)]['X_30'].mean()
mean_X_31 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_32 = train[(train['X_32']<1.7) & (train['X_32']>1)]['X_32'].mean()
mean_X_33 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_38 = train[(train['X_33']<0.7)]['X_33'].mean()

In [ ]:
train['X_30'] = train['X_30'].apply(lambda x : mean_X_30 if x >1.7 or x < 1 else x)
train['X_31'] = train['X_31'].apply(lambda x : mean_X_31 if x > 5 else x)
train['X_32'] = train['X_32'].apply(lambda x : mean_X_32 if x >1.7 or x < 1 else x)
train['X_33'] = train['X_33'].apply(lambda x : mean_X_33 if x > 5 else x)
train['X_38'] = train['X_38'].apply(lambda x : mean_X_38 if x > 0.7 else x)

In [ ]:

# 공차
train['gongcha_1_X'] = train[['X_38','X_39','X_40']].sum(axis = 1)
train['gongcha_2_X'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
train['all_sum_X'] = train.sum(axis=1)
train['all_mean_X'] = train.mean(axis=1)

### test 전처리

In [ ]:
test['max-min_X'] = test[['X_19','X_20','X_21','X_22']].max(axis=1) - test[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
test['slope_X'] = test[['X_38','X_39','X_40']].max(axis = 1) - test[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
test['SMT_X'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
test['X1~X6'] =  test[['X_01','X_06']].mean(axis=1)
test['X14~X18'] = test[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
test['X19~X22'] = test[['X_19','X_20','X_21','X_22']].mean(axis=1)
test['X24~X29'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
test['X30~X32'] = test[['X_30','X_32']].mean(axis=1)
test['X31~X33'] = test[['X_31','X_33']].mean(axis=1)
test['X38~X40'] = test[['X_38','X_39','X_40']].mean(axis=1)
test['X41~X45'] = test[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
test['X50~X56'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
test['con1_X'] = test['X_12'] - test['X_24']
test['con2_X'] = test['X_12'] - test['X_25']
test['con3_X'] = test['X_12'] - test['X_26']
test['con4_X'] = test['X_12'] - test['X_27']
test['con5_X'] = test['X_12'] - test['X_28']
test['con6_X'] = test['X_12'] - test['X_29']

In [ ]:
test['nd_area_X1'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X1'] = test['X_09']*test['X_07']/test['X_03']

test['nd_area_X2'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X2'] = test['X_09']*test['X_07']/test['X_03']

In [ ]:
# 무게 / 면적
test['X3/X7'] = test['X_03']/ test['X_07']

# 스크류 깊이 평균
test['X19X30'] = test[['X_19','X_30']].mean(axis=1)
test['X20X31'] = test[['X_20','X_31']].mean(axis=1)
test['X21X32'] = test[['X_21','X_32']].mean(axis=1)
test['X22X33'] = test[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
test['X14~X18_std'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
test['X19~X22_std'] = test[['X_19','X_20','X_21','X_22']].std(axis=1)
test['X24~X29_std'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

test['X38~X40_std'] = test[['X_38','X_39','X_40']].std(axis=1)

test['X41~X45_std'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
test['X50~X56_std'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
test['X14~X18_mid'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
test['X19~X22_mid'] = test[['X_19','X_20','X_21','X_22']].median(axis=1)
test['X24~X29_mid'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

test['X38~X40_mid'] = test[['X_38','X_39','X_40']].median(axis=1)

test['X41~X45_mid'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
test['X50~X56_mid'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
test['gongcha_1_X'] = test[['X_38','X_39','X_40']].sum(axis = 1)
test['gongcha_2_X'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
test['all_sum_X'] = test.sum(axis=1)
test['all_mean_X'] = test.mean(axis=1)

In [ ]:
train_x = train.filter(regex='X') # Input : X Featrue
train_y = train.filter(regex='Y') # Output : Y Feature

test_x = test.drop(columns=['ID'])

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(39607, 92)
(39607, 14)
(39608, 92)


# 모델 테스트

In [ ]:
from sklearn.metrics import mean_squared_error

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=0)

In [ ]:
model_lgb = MultiOutputRegressor(LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=700,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, random_state=42))

In [ ]:
model_lgb.fit(X_train, y_train)

MultiOutputRegressor(estimator=LGBMRegressor(bagging_fraction=0.8,
                                             bagging_freq=5, bagging_seed=9,
                                             feature_fraction=0.2319,
                                             feature_fraction_seed=9,
                                             learning_rate=0.05, max_bin=55,
                                             min_data_in_leaf=6,
                                             min_sum_hessian_in_leaf=11,
                                             n_estimators=700, num_leaves=5,
                                             objective='regression',
                                             random_state=42))

In [ ]:
pred_lgb_val = model_lgb.predict(X_val)

In [ ]:
y_val = np.array(y_val)
y_pred_val = np.array(pred_lgb_val)

In [ ]:
lg_nrmse(y_val, y_pred_val)

1.9707847770025864

In [ ]:
from sklearn.model_selection import KFold

# 앞에서 모델 구축한 후 진행 
# train_test_split하지 않고 train set 전체 사용

#폴드 세트 설정
kfold = KFold(n_splits=10) #5개로 split, 임의로 정할 수 있음 

#for문에서 train과 test데이터의 인덱스 추출
lg_nrmse_list = []
for train_index,test_index in kfold.split(train_x,train_y): #y를 안넣어도 됨
  X_train, X_val = train_x.iloc[train_index],train_x.iloc[test_index]
  y_train, y_val = train_y.iloc[train_index],train_y.iloc[test_index]
    
  #학습과 예측 수행 후 성능 반환
  model_lgb.fit(X_train,y_train)
  y_pred_val = model_lgb.predict(X_val)
  LG = lg_nrmse(np.array(y_val), np.array(y_pred_val))
  lg_nrmse_list.append(LG)
 

print(f" lg_nrmse 평균점수 : {np.array(lg_nrmse_list).mean():.7f}") 

# 제출

In [ ]:
model_gb = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=250, learning_rate=0.06,
                                   max_depth=7, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber')).fit(train_x, train_y)

In [ ]:
model_xgb = MultiOutputRegressor(xgb.XGBRegressor(olsample_bytree=0.8, silent=1, max_depth=7, learning_rate= 0.06, n_estimators=250,
                           random_state =7, nthread = -1, n_jobs=-1)).fit(train_x, train_y)

In [ ]:
import lightgbm as lgb

In [ ]:
model_lgb = MultiOutputRegressor(lgb.LGBMRegressor(objective='regression',n_estimators=250, bagging_fraction=0.7, learning_rate=0.06,
                         max_depth=7, subsample=0.7, feature_fraction=0.9, boosting_type='gbdt',
                         colsample_bytree=0.5, reg_lambda=5, n_jobs=-1)).fit(train_x, train_y)

In [ ]:
model_cat = MultiOutputRegressor(CatBoostRegressor(n_estimators=1300, learning_rate=0.03,
                                   max_depth=7)).fit(train_x, train_y)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
200:	learn: 0.7997468	total: 8.33s	remaining: 45.5s
201:	learn: 0.7996367	total: 8.37s	remaining: 45.5s
202:	learn: 0.7995607	total: 8.4s	remaining: 45.4s
203:	learn: 0.7995051	total: 8.44s	remaining: 45.3s
204:	learn: 0.7993910	total: 8.47s	remaining: 45.3s
205:	learn: 0.7993439	total: 8.51s	remaining: 45.2s
206:	learn: 0.7992668	total: 8.55s	remaining: 45.2s
207:	learn: 0.7991778	total: 8.59s	remaining: 45.1s
208:	learn: 0.7990996	total: 8.62s	remaining: 45s
209:	learn: 0.7990335	total: 8.67s	remaining: 45s
210:	learn: 0.7989365	total: 8.71s	remaining: 45s
211:	learn: 0.7988635	total: 8.75s	remaining: 44.9s
212:	learn: 0.7988088	total: 8.8s	remaining: 44.9s
213:	learn: 0.7987614	total: 8.83s	remaining: 44.8s
214:	learn: 0.7986885	total: 8.87s	remaining: 44.8s
215:	learn: 0.7985922	total: 8.91s	remaining: 44.7s
216:	learn: 0.7985361	total: 8.94s	remaining: 44.6s
217:	learn: 0.7984437	total: 8.99s	remaining: 44.6s
218:	learn: 0.7983584	total: 9.03s	r

In [ ]:
pred_gb = model_gb.predict(test_x)
pred_xgb = model_xgb.predict(test_x)
pred_lgb = model_lgb.predict(test_x)
pred_cat = model_cat.predict(test_x)

In [ ]:
ensemblelist = [[1,2,1,6],[2,1,2,5],[1,2,4,3],[4,3,2,1],[0,2,3,5],[0,0,2,8],[2,3,2,3],[1,2,3,4],[1,2,4,3],[1,5,3,1],[1,2,3,4],[2,1,6,1],[2,2,3,3],[3,1,3,3], [2.5,2.5,2.5,2.5],
                [2,2,4,2],[3,2,2,3]]

In [ ]:
count=0
for idx, col in enumerate(sample_submission.columns):
  if col=='ID':
    continue
  
  ensemble = pred_gb*(ensemblelist[count][0]/10) + pred_xgb*(ensemblelist[count][1]/10)+ pred_lgb*(ensemblelist[count][2]/10)+pred_cat*(ensemblelist[count][3]/10)
  sample_submission[col] = ensemble[:,idx-1]
  count=count+1
print('Done.')

sample_submission.to_csv('submit_ensemble_0820.csv', index=False)

Done.
